In [164]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
#import pyLDAvis.gensim_models
#from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cohan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [188]:
full_cv_df = pd.read_csv(r'C:\Users\Cohan\Documents\שבתון צחי\Resumes\DB3\DB3_cv.csv')
cv_df = full_cv_df.copy().iloc[0:200]

In [143]:
nlp = spacy.load('en_core_web_lg')

In [165]:
skills = r'C:\Users\Cohan\Documents\שבתון צחי\Resumes\jz_skill_patterns.jsonl'

In [166]:
ruler = nlp.add_pipe('entity_ruler')

ValueError: [E007] 'entity_ruler' already exists in pipeline. Existing names: ['tok2vec', 'tagger', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'entity_ruler', 'ner']

In [168]:
ruler.from_disk(skills)

In [169]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset

def unique_skills(x):
    return list(set(x))

In [204]:
def get_names(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            subset.append(ent.text)
    myset.append(subset)
    return subset

def unique_names(x):
    return list(set(x))

In [198]:
clean_cv_df = []
for i in range(cv_df.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        cv_df["CV"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean_cv_df.append(str(review))

In [199]:
cv_df["Clean_Resume"] = clean_cv_df

In [200]:
cv_df["skills"] = cv_df["Clean_Resume"].str.lower().apply(get_skills)
cv_df["skills"] = cv_df["skills"].apply(unique_skills)

In [205]:
cv_df["names"] = cv_df["Clean_Resume"].str.lower().apply(get_names)
cv_df["names"] = cv_df["names"].apply(unique_names)

In [206]:
cv_df.head()

,Category,Company,Job,Title,CV,Clean_Resume,skills,names
0,Administration,CDIB HK,Office Manager,Addleshaw_Treliza Li_Office Manager,\nPage 1 of 5 \nTreliza LI \nMobile : 9216 20...,page 1 5 treliza li mobile 9216 2087 tyfl com ...,"[monitoring, support, information management, ...","[vip head hr, amercian chinese, edward wildman..."
1,Administration,CDIB HK,Office Manager,Guard Capital_Godiva Tse_Office Manager,"Flat F, 5F, Block 5, Grand Palisades \n8 Shan ...",flat f 5f block 5 grand palisade 8 yin road ta...,"[support, project management, kong, compass, b...","[kwan godiva, tse, allan bedwick, cio coo, yin..."
2,Administration,CDIB HK,Office Manager,Lasalle IM_Cherish Chung_Admin EA,Cherish Chung \nTel: (852) 6209 5890 � E-mail:...,cherish chung tel 852 6209 5890 e mail cherish...,"[support, kong, business, accounting, security...","[chang pui chung memorial, mar 2013]"
3,Administration,CDIB HK,Office Manager,Paulson&Co_Ruby Shiu_Office Manager,EXPERIENCE : \nCURRENT EMPLOYMENT \nPaulson & ...,experience current employment paulson co inc h...,"[kong, business, ruby, database, mobile]","[james wong paulson co inc usa, intralinks sat..."
4,Administration,CDIB HK,Office Manager,Zhenghe Capital_Rainbow Chen_Office Manager,\nSelf-introduction and Career Expectation ...,self introduction career expectation 6 year ex...,"[support, kong, business, accounting, finance,...","[chen, qflp, dispatch bos]"


In [207]:
my_cv = nlp(cv_df["CV"].iloc[0])
displacy.render(my_cv, style="ent", jupyter=True)